In [ ]:
from pprint import pprint

import numpy as np
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff.forcefield import ForceField
from openff.toolkit.utils import get_data_file_path
from openff.units import unit
from simtk import openmm
from simtk import unit as simtk_unit

from openff.interchange.components.interchange import Interchange

In [ ]:
# Load a PDB file packaged with the OpenFF Toolkit
pdbfile = openmm.app.PDBFile(
    get_data_file_path("systems/packmol_boxes/propane_methane_butanol_0.2_0.3_0.5.pdb")
)

In [ ]:
# The OpenFF Topology currently requires that all molecular representations have
# chemiformatics data not present in a PDB file, including bond orders and
# stereochemistry. Therefore, Topology.from_openmm() requiures a list of
# Molecule objects with this data to be passed through. For more, see
# https://open-forcefield-toolkit.readthedocs.io/en/latest/api/generated/openff.toolkit.topology.Topology.html#openff.toolkit.topology.Topology.from_openmm)

molecules = [Molecule.from_smiles(smi) for smi in ["CCC", "C", "CCCCO"]]
topology = Topology.from_openmm(pdbfile.topology, unique_molecules=molecules)

In [ ]:
# Load in a mainline OpenFF force field
sage = ForceField("openff-2.0.0.offxml")

In [ ]:
# Create an Interchange object
out = Interchange.from_smirnoff(force_field=sage, topology=topology)

In [ ]:
# The OpenFF Topology represents a chemical graph without explicit positions, so
# set the positions of the Interchange object with the positions in the PDB fiile
out.positions = pdbfile.positions

In [ ]:
# Topology.from_openmm(), however, reads the periodic vectors from the PDB file and
# Interchnage.from_smirnoff() uses these to set the .box attribute, so we don't need
# to set it. But we can verify that they are equal
assert np.allclose(
    out.box.m_as(unit.nanometer),
    pdbfile.topology.getPeriodicBoxVectors().value_in_unit(simtk_unit.nanometer),
)

In [ ]:
# The Interchange package includes a module for obtaining single-point energies of
# Interchange objects by calling out to molecular mechanics engines. Here we query,
# inspect, and compare the energies obtained via OpenMM and GROMACS
from openff.interchange.drivers import (
    get_gromacs_energies,
    get_lammps_energies,
    get_openmm_energies,
)

gromacs_energies = get_gromacs_energies(out)
openmm_energies = get_openmm_energies(out)
lammps_energies = get_lammps_energies(out)

In [ ]:
print(openmm_energies)

In [ ]:
print(gromacs_energies)

In [ ]:
print(lammps_energies)

In [ ]:
pprint(openmm_energies - gromacs_energies)

In [ ]:
pprint(openmm_energies - lammps_energies)